In [1]:
import sys
!{sys.executable} -m pip install category_encoders


You should consider upgrading via the 'c:\Users\Nouran\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import category_encoders as ce
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import joblib

In [3]:
def data_preparation(df, label_column=None, scale_numeric=True, encode_categoricals=True, test_size=0.2, random_state=42):

  df['Blood Type'].isnull().sum()

  df['Insurance Provider'].fillna(df['Insurance Provider'].mode()[0], inplace=True)

  df['Admission Type'].fillna(df['Admission Type'].mode()[0], inplace=True)

  df['Hospital'].fillna(df['Hospital'].mode()[0], inplace=True)

  df['Billing Amount'].fillna(df['Billing Amount'].mean(), inplace=True)
  df['Blood Type'].fillna(df['Blood Type'].mode()[0], inplace=True)

  df['Name'] = df['Name'].str.title()

  df = pd.get_dummies(df, columns=['Gender'])

  df = pd.get_dummies(df, columns=['Blood Type'])

  df = pd.get_dummies(df, columns=['Medical Condition'])

  df['Date of Admission'] = pd.to_datetime(df['Date of Admission'], errors='coerce', dayfirst=True)

  df['Discharge Date'] = pd.to_datetime(df['Discharge Date'], errors='coerce', dayfirst=True)

  df['Doctor'] = df['Doctor'].str.title()

  df['Hospital'] = df['Hospital'].str.title()

  df = pd.get_dummies(df, columns=['Insurance Provider'])

  df = pd.get_dummies(df, columns=['Admission Type'])

  df = pd.get_dummies(df, columns=['Medication'])

  df['Billing_Is_Negative'] = df['Billing Amount'] < 0
  df['Billing Amount'] = df['Billing Amount'].abs()

  df['Billing_Is_Negative'] = df['Billing Amount'] < 0

  df['Billing Amount'] = df['Billing Amount'].abs()

  df['Name'] = df['Name'].str.strip()
  df['Name'] = df['Name'].str.replace(r'\s+', '_', regex=True)

  df['Doctor'] = df['Doctor'].str.strip()
  df['Doctor'] = df['Doctor'].str.replace(r'\s+', '_', regex=True)

  df['Hospital'] = df['Hospital'].str.strip()
  df['Hospital'] = df['Hospital'].str.replace(r'\s+', '_', regex=True)

  scaler = MinMaxScaler()

  billing_amount_values = df['Billing Amount'].values.reshape(-1, 1)

  normalized_billing_amount = scaler.fit_transform(billing_amount_values)

  df['Billing Amount'] = normalized_billing_amount

  scaler = MinMaxScaler()

  age_values = df['Age'].values.reshape(-1, 1)

  normalized_age = scaler.fit_transform(age_values)

  df['Age'] = normalized_age

  df = df.drop('Name', axis=1)

  df = df.drop('Room Number', axis=1)

  df['Days Spent'] = (df['Discharge Date'] - df['Date of Admission']).dt.days

  df = df.drop('Discharge Date', axis=1)

  df = df.drop('Date of Admission', axis=1)

  df = df.drop('Billing_Is_Negative', axis=1)

  scaler = MinMaxScaler()

  days_spent_values = df['Days Spent'].values.reshape(-1, 1)

  normalized_days_spent = scaler.fit_transform(days_spent_values)

  df['Days Spent'] = normalized_days_spent

  df = df.drop('Doctor', axis=1)

  df = df.drop('Hospital', axis=1)

  test_results_df = df[['ID']].copy()

  df = df.copy()
  df[df.select_dtypes(include='bool').columns] = df.select_dtypes(include='bool').astype(int)



  return df

In [4]:
df = pd.read_csv(r'D:\year 3\Semester 2\Machine Learning\Healthcare-test-results-classification-using-Machine-Learning\Original Dataset\test data.csv')
df.isnull().sum()

ID                      0
Name                    0
Age                     0
Gender                  0
Blood Type            308
Medical Condition       0
Date of Admission       0
Doctor                  0
Hospital                0
Insurance Provider      0
Billing Amount          0
Room Number             0
Admission Type          0
Discharge Date          0
Medication              0
dtype: int64

In [5]:
df.head()

,ID,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication
0,50001,KRISTa GiBsoN,60,Male,B-,Diabetes,08/04/2019,Bruce Garrett,Padilla-Walker,Cigna,1204.733872,226,Urgent,24/04/2019,Lipitor
1,50002,BRookE mCcUlLough,85,Male,A-,Cancer,14/12/2019,Scott Davis,"Nelson Walker Myers, and",Aetna,2177.616299,254,Emergency,25/12/2019,Penicillin
2,50003,DaNiEL THoMPSon,82,Female,A+,Cancer,14/02/2022,Kristin Martinez,Houston PLC,Cigna,8042.526388,131,Elective,07/05/2022,Penicillin
3,50004,nichOLas hunT,57,Male,B-,Asthma,08/04/2021,Mr. Luke Nunez Jr.,Hammond Ltd,Medicare,6060.230603,280,Elective,21/05/2021,Penicillin
4,50005,MElIssa mArTInEz,14,Male,AB-,Asthma,30/07/2022,Jacob Daniel,Morris-Arellano,Blue Cross,6902.620376,359,Urgent,09/09/2022,Lipitor


In [6]:
Final_df = data_preparation(df)

C:\Users\Nouran\AppData\Local\Temp\ipykernel_39332\2706223303.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Insurance Provider'].fillna(df['Insurance Provider'].mode()[0], inplace=True)
C:\Users\Nouran\AppData\Local\Temp\ipykernel_39332\2706223303.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values

In [7]:
df.isnull().sum()

ID                    0
Name                  0
Age                   0
Gender                0
Blood Type            0
Medical Condition     0
Date of Admission     0
Doctor                0
Hospital              0
Insurance Provider    0
Billing Amount        0
Room Number           0
Admission Type        0
Discharge Date        0
Medication            0
dtype: int64

In [8]:
Final_df.head()

,ID,Age,Billing Amount,Gender_Female,Gender_Male,Blood Type_A+,Blood Type_A-,Blood Type_AB+,Blood Type_AB-,Blood Type_B+,...,Insurance Provider_UnitedHealthcare,Admission Type_Elective,Admission Type_Emergency,Admission Type_Urgent,Medication_Aspirin,Medication_Ibuprofen,Medication_Lipitor,Medication_Paracetamol,Medication_Penicillin,Days Spent
0,50001,0.60,0.095294,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0.168317
1,50002,0.85,0.172366,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0.118812
2,50003,0.82,0.636985,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0.821782
3,50004,0.57,0.479947,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0.435644
4,50005,0.14,0.546682,0,1,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0.415842


In [9]:
#print the columns names
print(Final_df.columns)

Index(['ID', 'Age', 'Billing Amount', 'Gender_Female', 'Gender_Male',
       'Blood Type_A+', 'Blood Type_A-', 'Blood Type_AB+', 'Blood Type_AB-',
       'Blood Type_B+', 'Blood Type_B-', 'Blood Type_O+', 'Blood Type_O-',
       'Medical Condition_Arthritis', 'Medical Condition_Asthma',
       'Medical Condition_Cancer', 'Medical Condition_Diabetes',
       'Medical Condition_Hypertension', 'Medical Condition_Obesity',
       'Insurance Provider_Aetna', 'Insurance Provider_Blue Cross',
       'Insurance Provider_Cigna', 'Insurance Provider_Medicare',
       'Insurance Provider_UnitedHealthcare', 'Admission Type_Elective',
       'Admission Type_Emergency', 'Admission Type_Urgent',
       'Medication_Aspirin', 'Medication_Ibuprofen', 'Medication_Lipitor',
       'Medication_Paracetamol', 'Medication_Penicillin', 'Days Spent'],
      dtype='object')


In [10]:
model = joblib.load(r"C:\Users\Nouran\Downloads\xgb_model(1).pkl")


c:\Users\Nouran\AppData\Local\Programs\Python\Python310\lib\pickle.py:1718: UserWarning: [20:02:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\data\../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [11]:
# Drop the 'ID' column before prediction
features_for_prediction = Final_df.drop(columns=['ID'])

# Now pass only the required features
predictions = model.predict(features_for_prediction)


In [12]:
predictions = model.predict(features_for_prediction)

In [13]:
Final_df['Test Results'] = predictions

In [14]:
Final_df.head()

,ID,Age,Billing Amount,Gender_Female,Gender_Male,Blood Type_A+,Blood Type_A-,Blood Type_AB+,Blood Type_AB-,Blood Type_B+,...,Admission Type_Elective,Admission Type_Emergency,Admission Type_Urgent,Medication_Aspirin,Medication_Ibuprofen,Medication_Lipitor,Medication_Paracetamol,Medication_Penicillin,Days Spent,Test Results
0,50001,0.60,0.095294,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0.168317,1
1,50002,0.85,0.172366,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0.118812,0
2,50003,0.82,0.636985,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0.821782,0
3,50004,0.57,0.479947,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0.435644,1
4,50005,0.14,0.546682,0,1,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0.415842,2


In [15]:
submission_df = Final_df[['ID', 'Test Results']]

In [16]:
submission_df['Test Results'] = submission_df['Test Results'].replace({
    0: 'Normal',
    1: 'Inconclusive',
    2: 'Abnormal'
})

C:\Users\Nouran\AppData\Local\Temp\ipykernel_39332\3687894725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df['Test Results'] = submission_df['Test Results'].replace({


In [17]:
submission_df.head()

,ID,Test Results
0,50001,Inconclusive
1,50002,Normal
2,50003,Normal
3,50004,Inconclusive
4,50005,Abnormal


In [18]:
submission_df.to_csv('submission.csv', index=False)

In [19]:
# Display the first few rows of the submission DataFrame
print(submission_df.head())

      ID  Test Results
0  50001  Inconclusive
1  50002        Normal
2  50003        Normal
3  50004  Inconclusive
4  50005      Abnormal


In [20]:
#download the submission file
import shutil   
shutil.move('submission.csv', r'D:\year 3\Semester 2\Machine Learning\Healthcare-test-results-classification-using-Machine-Learning\Original Dataset\submission.csv')

'D:\\year 3\\Semester 2\\Machine Learning\\Healthcare-test-results-classification-using-Machine-Learning\\Original Dataset\\submission.csv'